In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pysr import PySRRegressor

## Importando os dados

In [5]:
grafeno = pd.read_csv('dados_RS/Neutral_Graphene_Oxide_Nanoflake_Dataset.csv')

##### Vamos trabalhar predizendo a relação entre a energia de Fermi e outras caracteristicas que o grafeno possui, e que estão presentes no banco de dados

In [6]:
grafeno = grafeno.select_dtypes(include=[np.number])

In [7]:
grafeno.fillna(grafeno.mean(), inplace=True)

In [8]:
for i in grafeno.columns:
    print(i)

charge_state
C
H
O
atom_number_total
C_concentration
H_concentration
O_concentration
avg_diameter
max_diameter
min_diameter
std_diameter
skew_diameter
kurt_diameter
anisotropy
area
AC_edge
ZZ_edge
total_edge
defects_count
defects_concentration
max_oop
mae_oop
std_oop
rmse_oop
residual_oop
ether_count
hydroxyl_count
carboxyl_count
edge_hydrogen_count
all_agent_group_count
ether_concentration
hydroxyl_concentration
carboxyl_concentration
def_local_ether_count
def_local_hydroxyl_count
def_local_carboxyl_count
def_local_other_count
max_bond_angle
max_bond_length
volume_per_atom
density_of_dangling_bonds
mass_density
particle_density
C-C:total_number
C-C:mean_value
C-C:error
C-C_sp1-sp1:total_number
C-C_sp1-sp1:mean_value
C-C_sp1-sp1:error
C-C_sp1-sp2:total_number
C-C_sp1-sp2:mean_value
C-C_sp1-sp2:error
C-C_sp1-sp3:total_number
C-C_sp1-sp3:mean_value
C-C_sp1-sp3:error
C-C_sp1-strained:total_number
C-C_sp1-strained:mean_value
C-C_sp1-strained:error
C-C_sp2-sp2:total_number
C-C_sp2-sp2:mean_

In [9]:
print(grafeno['electron_affinity'].apply(type).unique()) 

[<class 'float'>]


In [10]:
y = grafeno['Fermi_energy'].to_numpy()
X = grafeno[['total_energy', 'ionization_potential', 'electron_affinity', 'band_gap', 'electronegativity']].to_numpy()

In [11]:
model = PySRRegressor(
    maxsize=20,
    niterations=20,  # < Increase me for better results
    binary_operators=["+", "*"],
    unary_operators=[
        "cos",
        "exp",
        "sin",
        "inv(x) = 1/x",
        "square",
        # ^ Custom operator (julia syntax)
    ],
    extra_sympy_mappings={"inv": lambda x: 1 / x},
    # ^ Define operator for SymPy as well
    elementwise_loss="loss(prediction, target) = (prediction - target)^2",
    # ^ Custom loss function (julia syntax)
)


In [12]:
model.fit(X, y)

Compiling Julia backend...


c:\Users\isama\anaconda3\Lib\site-packages\pysr\sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
c:\Users\isama\anaconda3\Lib\site-packages\pysr\sr.py:2029: UserWarning: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (https://astroautomata.com/PySR/options/#batching). You should also reconsider if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form with symbolic regression. More datapoints will lower the search speed.
  warnings.warn(
[ Info: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (`options.batching`), and also if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is


Expressions evaluated per second: 8.710e+03
Head worker occupation: 9.3%
Progress: 20 / 300 total iterations (6.667%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
2           1.460e+01  7.971e+00  y = inv(-0.87158)
3           1.895e-01  4.345e+00  y = -0.09222 + x₄
4           1.771e-01  6.734e-02  y = inv(x₄) + x₄
6           8.138e-02  3.888e-01  y = sin(x₄) + (-0.93892 + x₄)
7           8.028e-02  1.359e-02  y = (sin(x₄) + sin(-1.8078)) + x₄
8           7.842e-02  2.341e-02  y = sin(inv(sin(x₄))) + (x₄ + -0.93892)
9           3.530e-02  7.982e-01  y = (sin(x₂) * cos(cos(sin(x₂)))) + -5.4213
13          3.445e-02  6.102e-03  y = sin(inv(inv(x₄)) + sin(cos(sin(x₃)))) + (x₄ + -0.93892)
15          3.434e-02  1.560e-03  y = sin(inv(inv(x₄)) + sin(cos(sin(x₄ * -0.93892)))) + (x₄ + -...
                                  0.93892)
16          3.219e-02  6.464e-02  y = sin(inv(inv

PySRRegressor.equations_ = [
	    pick     score                                           equation  \
	0         0.000000                                         -1.4451323   
	1         2.475744                            -2.4450593 + -2.5122995   
	2         0.716579                               sin(x2) + -5.5697436   
	3         0.174318                           sin(sin(x2)) + -5.508809   
	4         0.487787                  -5.313883 + sin(-0.96509665 + x4)   
	5         0.118585            sin(sin(x4 + -0.98830736)) + -5.2795835   
	6         0.057437          -5.250583 + sin(sin(sin(x4 + -1.011504)))   
	7   >>>>  0.326405       (sin(-0.9788418 + x4) * sin(x3)) + -5.252344   
	8         0.020008  (sin(x4 + -0.9937614) * sin(exp(cos(sin(x3))))...   
	9         0.078661  (square(sin(sin(x3) + 0.475989)) * sin(-1.0062...   
	10        0.002967  (sin(square(sin(sin(x3)) + 0.45118546)) * sin(...   
	11        0.013600  (sin(x4 + -0.9937614) * square(sin(exp(square(...   
	
	         loss  complexity  
	0   12.423559           1  
	1    0.087870           3  
	2    0.042918           4  
	3    0.036052           5  
	4    0.022135           6  
	5    0.019660           7  
	6    0.018563           8  
	7    0.013393           9  
	8    0.012613          12  
	9    0.011659          13  
	10   0.011624          14  
	11   0.011467          15  
]